# Assignment 2 

## Kun Huang, Yizhou Jin

In this assignment, we try some features in **TensorFlow** and **gym**. We tried to rewrite the example in a tensorflow style, but since it involves **gym**, we have not figured out how to rewrite it completely.   

First we import all the libraries necessary. 

In [8]:
import gym
import numpy as np
import random
import tensorflow as tf
from IPython.display import clear_output
import time
import warnings

warnings.filterwarnings('ignore')

In [2]:
#Set environment in gym
env = gym.make('FrozenLake-v0')

## Codes 1

The idea mainly comes from [Simple Reinforcement Learning with Tensorflow Part 0: Q-Learning with Tables and Neural Networks](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0). It uses the idea of neural networks, that is, it uses a one-layer network which takes the state encoded in a vector (1x16,i.e. `inputs1`), and produces a vector of 4 Q-values (i.e. `Qout` and `nextQ`). 

In [6]:
tf.reset_default_graph()

#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.1))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

# loss, L2
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

Next, we run the codes rewritten in tensorflow.

In [7]:
# set a initializer for tensor
init = tf.global_variables_initializer()

# Set learning parameters
gamma = .7
episodes = 5000
alpha = 0.2
total = 0
with tf.Session() as sess:
    sess.run(init)
    for i in range(episodes):
        #Reset environment and get first new observation
        state = env.reset()
        done = False
        epsilon = 1./(i+1)
        #The Q-Network
        while not done:
            #Choose an action by greedily (with e chance of random action) from the Q-network
            action,allQ = sess.run([predict,Qout], feed_dict={inputs1:np.identity(16)[state:state + 1]})
            targetQ = allQ
            if random.uniform(0,1) < epsilon:
                action[0] = env.action_space.sample()
            else:
                action[0] = int(np.argmax(targetQ))
            #Get new state and reward from environment
            new_state, reward, done, info = env.step(action[0])
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout, feed_dict={inputs1:np.identity(16)[new_state:new_state + 1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ[0,action[0]] = alpha * (reward + gamma*maxQ1 - targetQ[0, action[0]])
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[state:state+1],nextQ:targetQ})
            total += reward
            state = new_state
print ("Average rewards: " + str(total/episodes))
print("Learned Q-Table")
print(W1)


Average rewards: 0.0204
Learned Q-Table
[[4.07298430e-05 3.75570817e-05 2.07737670e-04 4.03624836e-05]
 [6.23325686e-05 6.50726433e-05 4.59288387e-03 5.98590268e-05]
 [1.82495016e-04 5.35416533e-04 1.93211803e-04 1.53970104e-04]
 [5.09598292e-03 2.24781426e-04 2.72362260e-04 2.54953018e-04]
 [2.18543151e-04 2.59502383e-04 3.27463960e-03 2.51325517e-04]
 [9.84402895e-02 6.33234754e-02 4.54147831e-02 7.81726763e-02]
 [2.56862491e-03 8.44424963e-03 2.44291290e-03 2.62246816e-03]
 [5.62756658e-02 3.01493891e-02 8.67512971e-02 8.21112171e-02]
 [3.14887002e-04 2.77046929e-04 2.58776080e-03 3.15808575e-04]
 [7.89283251e-04 7.68002006e-04 2.25966098e-03 7.90609687e-04]
 [1.56975293e-03 1.63703004e-03 6.31615473e-03 1.81776949e-03]
 [2.21647974e-02 5.98871484e-02 1.26037477e-02 1.89152490e-02]
 [9.79747176e-02 1.72407757e-02 7.40739098e-03 1.09730959e-02]
 [2.16108514e-03 2.42753327e-03 2.33033509e-03 5.12008183e-03]
 [1.02388635e-02 9.29095075e-02 1.12937074e-02 9.47304070e-03]
 [2.28876825e-0

## Code2

Similarly, we rewrite the sample code into tensorflow.

In [9]:
env.reset()
episodes = 10
max_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for ep in range(episodes):
        state = env.reset()
        step = 0
        done = False
        clear_output()
        print("*****************")
        print("Episodes", ep)
        print("*****************")
        for step in range(max_steps):
            action,allQ = sess.run([predict,Qout], feed_dict={inputs1:np.identity(16)[state:state + 1]})
            action[0] = np.argmax(allQ)
            new_state, reward, done, info = env.step(action[0])
            if done:
                env.render()
                print("*****************")
                print("Steps", step)
                print("*****************")
                time.sleep(2)
                break
        state = new_state
env.close()


*****************
Episodes 9
*****************
  (Left)
SFFF
FHFH
FFFH
HFFG
*****************
Steps 9
*****************
